In [4]:
import numpy as np
from collections import deque
import resource
import time
import heapq
import psutil
import os


class Node: #will contain a whole 3x3 matrix
    def __init__(self, state, parent, action, path_cost=0):
        self.state = state
        self.parent = parent
        self.action = action
        self.path_cost = path_cost

    def __lt__(self, other):
        return self.path_cost < other.path_cost

class StackFrontier:
    def __init__(self):
        self.frontier = []

    def add(self, node):
        self.frontier.append(node)

    def contains_state(self, state):
        return any((node.state == state).all() for node in self.frontier)

    def empty(self):
        return len(self.frontier) == 0

    def remove(self):
        if self.empty():
            raise Exception("Empty Frontier")
        else:
            node = self.frontier[-1]
            self.frontier = self.frontier[:-1]
            return node

class QueueFrontier(StackFrontier):
    def remove(self):
        if self.empty():
            raise Exception("Empty Frontier")
        else:
            node = self.frontier[0]
            self.frontier = self.frontier[1:]
            return node

class PriorityQueue:
    def __init__(self):
        self.heap = []

    def add(self, node):
        heapq.heappush(self.heap, node) #ensures that the smallest element is always at the root

    def pop(self):
        return heapq.heappop(self.heap) #returns the smallest node (based on path cost), it uses Node class's __It__ method by default

    def empty(self):
        return len(self.heap) == 0

class Puzzle:
    def __init__(self, start, goal): #attributes of the puzzle object
        self.start = start
        self.goal = goal
        self.solution = None
        self.num_explored = 0
        self.memory_consumed = 0
        self.time_taken = 0

    def neighbors(self, state):
        mat = state
        row, col = np.argwhere(mat == 0)[0]  # Get the position of the blank space
        results = []

        if row > 0: #means the blank space can be moved upwards
            mat1 = np.copy(mat)
            mat1[row][col] = mat1[row - 1][col] #swap the number with 0
            mat1[row - 1][col] = 0
            results.append(('up', mat1)) #store the action
        if col > 0:#means the blank space can be moved left
            mat1 = np.copy(mat)
            mat1[row][col] = mat1[row][col - 1]
            mat1[row][col - 1] = 0
            results.append(('left', mat1))
        if row < 2: #means the blank space can be moved down
            mat1 = np.copy(mat)
            mat1[row][col] = mat1[row + 1][col]
            mat1[row + 1][col] = 0
            results.append(('down', mat1))
        if col < 2: #means the blank space can be moved right
            mat1 = np.copy(mat)
            mat1[row][col] = mat1[row][col + 1]
            mat1[row][col + 1] = 0
            results.append(('right', mat1))

        return results #return the action taken and the updated matrix

    def print_solution(self):
        solution = self.solution if self.solution is not None else None
        print("Start State:\n", self.start, "\n")
        print("Goal State:\n",  self.goal, "\n")
        print("Time taken: {:.10f} seconds".format(self.time_taken))
        print("Path Cost:", len(solution[0]) if solution is not None else "N/A")
        #print("Memory Consumed:", self.memory_consumed, "kilobytes")
        print("No of Node Visited:", self.num_explored)
        if solution:
            print("\nSolution:\n ")
            for action, cell in zip(solution[0], solution[1]):
                print("action: ", action, "\n", cell, "\n")
            print("Goal Reached!!")


    def solve_bfs(self):
        start_time = time.time() #keep the time recorded in a variable to output later on

        start = Node(state=self.start, parent=None, action=None) # a node object is created, parent = none because it has no parent (it is a starting point), similarly action = none
        frontier = QueueFrontier() #queue object is made and used for FIFO purpose
        frontier.add(start) #the initial node is added to the queue

        self.explored = [] #initliazes and empty array within the Puzzle class accesible by every object of the class

        while True:
            if frontier.empty():
                raise Exception("No solution") #if no nodes available means no solution

            node = frontier.remove() #returns the first node in the queue
            self.num_explored += 1 #increments the number of nodes explored

            if (node.state == self.goal).all(): #check if the current state and goal state are equal means our objective reached
                actions = []
                cells = []
                while node.parent is not None: #backtracking hence do until we don't reach the initial state
                    actions.append(node.action)
                    cells.append(node.state)
                    node = node.parent
                actions.reverse() #since we were backtracking, the list is reversed in the end to make it go from initial -> final
                cells.reverse()
                self.solution = (actions,  cells) #the solution is assigned to the bfs_solver object attribute
                self.time_taken = time.time() - start_time #calculate the time taken and assign to attribute
                self.memory_consumed = psutil.Process(os.getpid()).memory_info().rss / float(2 ** 20) # Memory usage in MB #recieve the memory consumed in the process
                return #work is finished, return to main

            #otherwise if the current state != goal state
            self.explored.append(node.state) #add the current state to the explored list

            for action, state in self.neighbors(node.state): #iterate over the neighboring states of the current state, return neighboring states and the actions required to reach them
                if not frontier.contains_state(state): #check if the recieved neighboring state is not already in frontier queue
                    child = Node(state=state, parent=node, action=action) #a new object child is made to represent this state
                    frontier.add(child) # the child is added to the frontier

    def solve_dfs(self):
        start_time = time.time()
        self.num_explored += 1

        def dfs(node, visited_states):
            if (node.state == self.goal).all():
                # Goal state reached, backtrack to find solution
                actions = []
                cells = []
                while node.parent is not None:
                    actions.append(node.action)
                    cells.append(node.state)
                    node = node.parent
                actions.reverse()
                cells.reverse()
                self.solution = (actions, cells)
                return True

            for action, state in self.neighbors(node.state):
                child = Node(state=state, parent=node, action=action)
                if tuple(child.state.flatten()) not in visited_states:
                    self.num_explored += 1
                    visited_states.add(tuple(child.state.flatten()))
                    if dfs(child, visited_states):
                        return True

            return False

        start_node = Node(state=self.start, parent=None, action=None)
        visited_states = set()
        visited_states.add(tuple(start_node.state.flatten()))
        if dfs(start_node, visited_states):
            self.time_taken = time.time() - start_time
            self.memory_consumed = psutil.Process(os.getpid()).memory_info().rss / float(2 ** 20) # Memory usage in MB
        else:
            raise Exception("No solution found")

    def IDS_dfs(self, node, depth_limit):
            if depth_limit == 0:
                return None
            if (node.state == self.goal).all():
                actions = []
                cells = []
                while node.parent is not None:
                    actions.append(node.action)
                    cells.append(node.state)
                    node = node.parent
                actions.reverse()
                cells.reverse()
                self.solution = (actions, cells)
                return node
            for action, state in self.neighbors(node.state):
                child = Node(state=state, parent=node, action=action)
                self.num_explored += 1
                result = self.IDS_dfs(child, depth_limit - 1)
                if result:
                    return result
            return None

    def solve_ids(self):
            start_time = time.time()
            depth_limit = 0 #since IDS works by incrementing depth limit each time
            while True:
                result = self.IDS_dfs(Node(state=self.start, parent=None, action=None), depth_limit)
                if result:
                    self.time_taken = time.time() - start_time
                    self.memory_consumed = resource.getrusage(resource.RUSAGE_SELF).ru_maxrss
                    return
                depth_limit += 1

    def solve_uniform_cost_search(self):
        start_time = time.time()

        start_node = Node(state=self.start, parent=None, action=None, path_cost=0)
        frontier = PriorityQueue()
        frontier.add(start_node)

        explored = set()

        while not frontier.empty():
            node = frontier.pop()
            self.num_explored += 1

            if (node.state == self.goal).all():
                actions = []
                cells = []
                path_costs = []
                while node.parent is not None:
                    actions.append(node.action)
                    cells.append(node.state)
                    path_costs.append(node.path_cost)
                    node = node.parent
                actions.reverse()
                cells.reverse()
                path_costs.reverse()
                self.solution = (path_costs, actions, cells)
                self.time_taken = time.time() - start_time
                self.memory_consumed = resource.getrusage(resource.RUSAGE_SELF).ru_maxrss
                return

            explored.add(tuple(node.state.flatten()))

            for action, state in self.neighbors(node.state):
                child = Node(state=state, parent=node, action=action, path_cost=node.path_cost + 1)
                if tuple(child.state.flatten()) not in explored:
                    frontier.add(child)


def input_state(prompt):
    print(prompt)
    state = []
    for _ in range(3):
        row = list(map(int, input().split()))
        state.append(row)
    return np.array(state)




#this function takes input row by row for the start and goal state
def input_state(prompt):
    print(prompt)
    state = []
    for i in range(3):
        row = list(map(int, input().split()))
        state.append(row)
    return np.array(state) #returns a 3x3 matrix

# Input initial and goal states
start = input_state("Enter the initial state (3x3 grid with numbers 0-8):")
goal = input_state("Enter the goal state (3x3 grid with numbers 0-8):")

print()

# Call BFS method
bfs_solver = Puzzle(start, goal) #a 'Puzzel' object named bfs_solver is created, which is passed the start and goal state matrix
memory_before_bfs = psutil.Process(os.getpid()).memory_info().rss / 1024**2  # Memory before BFS  # Memory before BFS
bfs_solver.solve_bfs() #now  that the object is created, the bfs function is called for the object we made
memory_after_bfs = bfs_solver.memory_consumed
print("Memory Consumed (BFS): {:.2f} MB".format(memory_after_bfs))
print("======================================")
print("Breadth First Search (BFS): ")
bfs_solver.print_solution()
print("======================================")

#print()

# Call DFS method
dfs_solver = Puzzle(start, goal)
memory_before_dfs = psutil.Process(os.getpid()).memory_info().rss / 1024**2
dfs_solver.solve_dfs()
memory_after_dfs = dfs_solver.memory_consumed
print("Memory Consumed (DFS): {:.2f} MB".format(memory_after_dfs))
print("======================================")
print("Depth First Search (DFS): ")
dfs_solver.print_solution()
print("======================================")

print()
# Call IDS method
ids_solver = Puzzle(start, goal)
memory_before_ids = psutil.Process(os.getpid()).memory_info().rss / 1024**2
ids_solver.solve_ids()
memory_after_ids = ids_solver.memory_consumed
print("Memory Consumed (IDS): {:.2f} MB".format(memory_after_ids))
print("======================================")
print("Iterative Deepening Search (IDS):")
ids_solver.print_solution()
print("======================================")

print()

# Call Uniform Cost Search method
ucs_solver = Puzzle(start, goal)
memory_before_ucs = psutil.Process(os.getpid()).memory_info().rss / 1024**2
ucs_solver.solve_uniform_cost_search()
memory_after_ucs = ucs_solver.memory_consumed
print("Memory Consumed (UCS): {:.2f} MB".format(memory_after_ucs))
print("======================================")
print("Uniform Cost Search (UCS):")
ucs_solver.print_solution()
print("======================================")




Enter the initial state (3x3 grid with numbers 0-8):
1 2 0
3 4 5
6 7 8
Enter the goal state (3x3 grid with numbers 0-8):
0 1 2
3 4 5
6 7 8

Memory Consumed (BFS): 111.37 MB
Breadth First Search (BFS): 
Start State:
 [[1 2 0]
 [3 4 5]
 [6 7 8]] 

Goal State:
 [[0 1 2]
 [3 4 5]
 [6 7 8]] 

Time taken: 0.0005435944 seconds
Path Cost: 2
No of Node Visited: 4

Solution:
 
action:  left 
 [[1 0 2]
 [3 4 5]
 [6 7 8]] 

action:  left 
 [[0 1 2]
 [3 4 5]
 [6 7 8]] 

Goal Reached!!
Memory Consumed (DFS): 111.37 MB
Depth First Search (DFS): 
Start State:
 [[1 2 0]
 [3 4 5]
 [6 7 8]] 

Goal State:
 [[0 1 2]
 [3 4 5]
 [6 7 8]] 

Time taken: 0.0002114773 seconds
Path Cost: 2
No of Node Visited: 3

Solution:
 
action:  left 
 [[1 0 2]
 [3 4 5]
 [6 7 8]] 

action:  left 
 [[0 1 2]
 [3 4 5]
 [6 7 8]] 

Goal Reached!!

Memory Consumed (IDS): 114044.00 MB
Iterative Deepening Search (IDS):
Start State:
 [[1 2 0]
 [3 4 5]
 [6 7 8]] 

Goal State:
 [[0 1 2]
 [3 4 5]
 [6 7 8]] 

Time taken: 0.0003499985 secon